In [35]:
import pandas 
import sklearn
import math
import numpy as np
import random
random.seed(12)

# Cultures

In [36]:
class Сulture:
    def __init__(self, n, K):
        self.n = n
        self.K = n
        self.ctype = None
    
    def inequalityMeasure(self, k1, k2):
        return 0

In [68]:
class Rousseauist_culture(Сulture):
    # We assume that 1 > 2 > ... > K
    def setParams(self, alpha, beta):
        # alpha >= 0
        # beta >= 0
        self.alpha = alpha
        self.beta = beta
        self.ctype = "Rousseauist_culture"
        
    def createMatrix(self):
        M = np.zeros((self.K, self.K))
        for t in range(self.n):
            M += self.createMatrixProfile()
        return M
        
    def createMatrixProfile(self):
        tempM = np.zeros((self.K, self.K))
        for i in range(self.K):
            for j in range(i+1, self.K):
                if( random.random() < self.getP(i,j)):
                    tempM[i, j] += 1
                else:
                    tempM[j, i] += 1
        return tempM
        
    def getP(self, k1, k2):
        # Truchon and Drissi-Bakhkhat
        
        # k > k'
        if(k2 <= k1): 
            raise NameError("k >= k'")
            
        s = math.exp(self.alpha + self.beta*(k2-k1))
        return s/(1+s)
        
    def inequalityMeasure(self, k1, k2):
        # Truchon and Drissi-Bakhkhat
        return 0    
    
        
class Impartial_culture(Сulture):
    def setParams(self):
        self.ctype = "Impartial_culture"
        
    def inequalityMeasure(self, k1, k2):
        # symmetry ???
        return 0
    
    def createLinearProfile(self):
        profile = list(range(self.K))
        random.shuffle(profile)
        return profile
    
    def linearIntoMatrix(self, profile):
        tempM = np.zeros((self.K, self.K))
        for i in range(self.K):
            for j in range(i+1, self.K):
                tempM[profile[i], profile[j]] += 1
        return tempM

class Distributive_culture(Сulture):
    def setParams(self):
        self.ctype = "Distributive_culture"
        
    def inequalityMeasure(self, k1, k2):
        # Gini index
        return 0
        
class Spatial_culture(Сulture):
    def setDimention(self, d):
        self.d = d
        self.ctype = "Spatial_culture"
        
    def inequalityMeasure(self, k1, k2):
        # ??? 
        return 0

In [70]:
r = Rousseauist_culture(5,6)
r.setParams(1,1)
r.getP(4,5)
print(r.createMatrix())

i = Impartial_culture(5,6)
i.setParams()
profile = i.createLinearProfile()
print(profile)
print(i.linearIntoMatrix(profile))

[[0. 5. 5. 5. 5.]
 [0. 0. 5. 5. 5.]
 [0. 0. 0. 5. 5.]
 [0. 0. 0. 0. 5.]
 [0. 0. 0. 0. 0.]]
[3, 0, 4, 1, 2]
[[0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 0.]]


In [47]:
def generateCultureFile(culture: Culture):
    if culture.ctype == "Rousseauist_culture":
        pass
    elif culture.ctype == "Impartial_culture":
        pass
    elif culture.ctype == "Distributive_culture":
        pass
    elif culture.ctype == "Spatial_culture":
        pass
    

# Voting types

In [ ]:

class Voting:
    def __init__(self, filepath, method):
        self.df = pd.read_csv(filepath)
        self.df = self.df.drop('Same', axis=1)
        self.alternatives = set(self.df.columns)
        self.method = method

        # All preferences for each alternative
        self.ALTR_dict = {} # was profile_dict
        for cand in self.alternatives:
            self.ALTR_dict[cand] = self.df[cand].values

        # Preferences for each voter
        self.INDV_d = {}
        for itr, row in self.df.iterrows():
            self.INDV_d[itr]=row.to_dict()

    def voting(self):
        if (self.method == 'Plurality'):
            return self.plurality_voting()
        elif (self.method == 'Borda'):
            return self.borda_voting()
        elif (self.method == 'Nanson'):
            return self.nanson_voting()
        elif (self.method == 'STV'):
            return self.stv_voting()
        elif (self.method == 'Copeland'):
            return self.copeland_voting()
        elif (self.method == 'OurMethod'):
            return self.our_voting()


    # Common fuctions:
    def reboot(self):
        # Normalize
        self.df = self.normalize_df(self.df)
        self.alternatives = set(self.df.columns)

        self.ALTR_dict = {}
        for cand in self.alternatives:
            self.ALTR_dict[cand] = self.df[cand].values

        # Preferences for each voter
        self.INDV_d = {}
        for itr, row in self.df.iterrows():
            self.INDV_d[itr]=row.to_dict()

        return self.df

    def normalize_df(self, df):
        new_df = pd.DataFrame(columns=df.columns)
        for itr, row in df.iterrows():
            new_row = {}
            sorted_dict = sorted(row.to_dict().items(), key=operator.itemgetter(1))
            for i, el in enumerate(sorted_dict):
                new_row[el[0]] = i + 1
            new_df = new_df.append(new_row, ignore_index=True)
        return new_df

    def lexicografic_order(self, winners):
        winners.sort()
        return winners[0]

    def get_winners(self, score_dict):
        winners = []
        max_val = max([i for i in score_dict.values()])
        for key in score_dict.keys():
            if(score_dict[key]==max_val):
                winners.append(key)
        return winners

    def get_losers(self, score_dict):
        losers = []
        if(self.method == 'Nanson'):
            Mean = np.mean([score_dict[cand] for cand in score_dict.keys()])
            for cand in self.alternatives:
                if(score_dict[cand]<Mean):
                    self.df = self.df.drop(cand, axis=1)
        elif(self.method == 'STV'):
            min_val = min([i for i in score_dict.values()])
            for key in score_dict.keys():
                if(score_dict[key]==min_val):
                    losers.append(key)
        elif (self.method == 'OurMethod'):
            max_val = max([i for i in score_dict.values()])
            for key in score_dict.keys():
                if (score_dict[key] == max_val):
                    losers.append(key)
        return losers

    def checkCondorset(self, show=False):
        for el1 in self.alternatives:
            Winner_Flag = True
            for el2 in self.alternatives:
                if(el1!=el2):
                    scores = [0, 0]
                    for itr, row in self.df.iterrows():
                        if(self.df[el1][itr]<self.df[el2][itr]):
                            # el1 - winner
                            scores[0] += 1
                        else:
                            scores[1] += 1
                    if(scores[0]>scores[1]):
                        pass
                    elif (scores[0] < scores[1]):
                        condorset_winner = None
                        Winner_Flag = False
                        break
            if(Winner_Flag):
                if(show):
                    print(f'Alternative {el1} is a Condorcet winner.')
                self.df = self.df.drop(el1, axis=1)
                self.df = self.reboot()
                return True
        return False

    def deleteCondorset(self, show=False):

        Cond_Win = self.checkCondorset(show)
        while (Cond_Win):
            Cond_Win = self.checkCondorset(show)

    # Plurality:
    def plurality_score(self, cand):
        return sum([el == 1 for el in self.ALTR_dict[cand]])

    def plurality_voting(self):
        score_dict = {}
        for cand in self.alternatives:
            score_dict[cand] = self.plurality_score(cand)
        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # Borda:
    def pref_into_borda(self, alternative_prefs, m):
        res_list = []
        for pref in alternative_prefs:
            res_list.append(m-pref)
        return res_list

    def borda_score(self, cand, m):
        return sum(self.pref_into_borda(self.ALTR_dict[cand], m))

    def borda_voting(self):
        score_dict = {}
        m = len(self.alternatives)
        for cand in self.alternatives:
            score_dict[cand] = self.borda_score(cand, m)
        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # Nanson
    def nanson_voting(self):

        score_dict = {}
        m = len(self.alternatives)

        for cand in self.alternatives:
            score_dict[cand] = self.borda_score(cand, m)

        winners = self.get_winners(score_dict)
        while(len(winners)!=len(self.alternatives)):
            losers = self.get_losers(score_dict)
            for loser in losers:
                self.df = self.df.drop(loser, axis=1)

            self.df = self.reboot()
            m = len(self.alternatives)

            score_dict = {}
            for cand in self.alternatives:
                score_dict[cand] = self.borda_score(cand, m)

            winners = self.get_winners(score_dict)

        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # STV
    def stv_score(self, cand):
        return sum([el == 1 for el in self.ALTR_dict[cand]])

    def stv_voting(self):
        # Get scores
        score_dict = {}
        for cand in self.alternatives:
            score_dict[cand] = self.stv_score(cand)
        # Check two or more winners
        winners = self.get_winners(score_dict)
        while(len(winners)!=len(self.alternatives)):
            losers = self.get_losers(score_dict)
            for loser in losers:
                self.df = self.df.drop(loser, axis=1)
            # Normalize
            self.df = self.reboot()

            score_dict = {}
            for cand in self.alternatives:
                score_dict[cand] = self.stv_score(cand)

            winners = self.get_winners(score_dict)

        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # Copeland
    def build_dominance_graph(self):
        graph = {}
        reverse_graph = {}
        for cand in self.alternatives:
            graph[cand] = set()
            reverse_graph[cand] = set()

        for el1 in self.alternatives:
            for el2 in self.alternatives:
                if (el1 != el2):
                    scores = [0, 0]
                    for itr, row in self.df.iterrows():
                        if (self.df[el1][itr] < self.df[el2][itr]):
                            # el1 - winner
                            scores[0] += 1
                        else:
                            scores[1] += 1
                    if (scores[0] > scores[1]):
                        graph[el1].add(el2)
                        reverse_graph[el2].add(el1)
                    elif (scores[0] < scores[1]):
                        graph[el2].add(el1)
                        reverse_graph[el1].add(el2)

        return graph, reverse_graph

    def copeland_score(self, cand):
        return len(self.graph[cand])

    def copeland_voting(self):
        self.graph, self.reversed_graph = self.build_dominance_graph()
        score_dict = {}
        for cand in self.alternatives:
            score_dict[cand] = self.copeland_score(cand)
        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

